## 2유형 - 모델링 및 평가

### 회귀분석 (y값 = 종속변수가 연속형 변수) = Regression
### 분류분석 (y값 = 종속변수가 범주형 변수) = Classifier

### 현재까지 지도학습만 나옴
#### - macro f1 score
#### - RMSE
#### - MSE
#### - accuracy_score
#### - AUC

##### 2유형의 가장 중요한 부분은 적절한 예측값을 제출하는 것 : 성능평가는 출제기관에서 진행
##### 적절한 예측값을 제출하기 위해서는 train / validataion set으로 나오고, validataion set을 가지고 성능평가를 해봐야 함
##### from sklearn.model_selection import train_test_split

### 분석 순서
#### x_train / y_train 으로 모델 학습
#### x_val로 y_pred 구하기
#### y_val로 실제 예측값 구하고 y_pred와 값 비교

### 2유형 summary
### 2유형 분석 순서
#### 1. 라이브러리 및 데이터 확인
#### 2. 데이터 탐색(EDA) - shape, info(), value_counts(), describe()
#### 3. 데이터 전처리 및 분리
##### - 결측치(대체 or 삭제), 이상치(대체 or 삭제), 변수 처리(범주형 변수의 경우 원핫인코딩 필요, pd.get_dummies()
#### 4. 모델링 및 성능 평가
#### 5. 예측값 제출

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [21]:
from sklearn.datasets import load_wine
# 와인 datasets load
wine = load_wine()
x = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.DataFrame(wine.target)

# 실기 시험 데이터셋으로 셋팅하기 (수정 금지)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=2023)

x_test = pd.DataFrame(x_test)
x_train = pd.DataFrame(x_train)
y_train = pd.DataFrame(y_train)

x_test.reset_index()
y_train.columns = ['target']

In [6]:
print(x_train.head(3))
print(x_test.head(3))
print(y_train.head(3))

     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
52     13.82        1.75  2.42               14.0      111.0           3.88   
146    13.88        5.04  2.23               20.0       80.0           0.98   
44     13.05        1.77  2.10               17.0      107.0           3.00   

     flavanoids  nonflavanoid_phenols  proanthocyanins  color_intensity   hue  \
52         3.74                  0.32             1.87             7.05  1.01   
146        0.34                  0.40             0.68             4.90  0.58   
44         3.00                  0.28             2.03             5.04  0.88   

     od280/od315_of_diluted_wines  proline  
52                           3.26   1190.0  
146                          1.33    415.0  
44                           3.35    885.0  
     alcohol  malic_acid   ash  alcalinity_of_ash  magnesium  total_phenols  \
168    13.58        2.58  2.69               24.5      105.0           1.55   
144    12.25        

In [4]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(142, 13)
(142, 1)
(36, 13)


In [5]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
Index: 142 entries, 52 to 115
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alcohol                       142 non-null    float64
 1   malic_acid                    142 non-null    float64
 2   ash                           142 non-null    float64
 3   alcalinity_of_ash             142 non-null    float64
 4   magnesium                     142 non-null    float64
 5   total_phenols                 142 non-null    float64
 6   flavanoids                    142 non-null    float64
 7   nonflavanoid_phenols          142 non-null    float64
 8   proanthocyanins               142 non-null    float64
 9   color_intensity               142 non-null    float64
 10  hue                           142 non-null    float64
 11  od280/od315_of_diluted_wines  142 non-null    float64
 12  proline                       142 non-null    float64
dtypes: float6

In [15]:
print(x_train.describe())
print(x_test.describe())
print(y_train.describe())

          alcohol  malic_acid         ash  alcalinity_of_ash   magnesium  \
count  142.000000  142.000000  142.000000         142.000000  142.000000   
mean    13.025915    2.354296    2.340211          19.354225   98.732394   
std      0.812423    1.142722    0.279910           3.476825   13.581859   
min     11.030000    0.740000    1.360000          10.600000   70.000000   
25%     12.370000    1.610000    2.190000          16.800000   88.000000   
50%     13.050000    1.820000    2.320000          19.300000   97.000000   
75%     13.685000    3.115000    2.510000          21.500000  106.750000   
max     14.830000    5.800000    3.230000          30.000000  151.000000   

       total_phenols  flavanoids  nonflavanoid_phenols  proanthocyanins  \
count     142.000000  142.000000            142.000000       142.000000   
mean        2.303592    2.043592              0.361479         1.575070   
std         0.633955    1.033597              0.124627         0.576798   
min         0.9

In [16]:
print(y_train.head())
print(y_train.value_counts())

     target
52        0
146       2
44        0
67        1
43        0
target
1         57
0         47
2         38
Name: count, dtype: int64


In [17]:
# 데이터 결측치, 이상치, 변수
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int64
alcohol                         0
malic_acid                      0
ash                             0
alcalinity_of_ash               0
magnesium                       0
total_phenols                   0
flavanoids                      0
nonflavanoid_phenols            0
proanthocyanins                 0
color_intensity                 0
hue                             0
od280/od315_of_diluted_wines    0
proline                         0
dtype: int64
target    0
dtype: int64


In [22]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['target'], test_size=0.2, stratify=y_train['target'], random_state=2023)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(113, 13)
(29, 13)
(113,)
(29,)


In [23]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

In [24]:
y_pred = model.predict(x_val)
print(y_pred)

[2 2 1 0 0 2 0 1 2 0 0 0 1 1 2 2 1 1 1 0 1 1 1 2 1 1 2 0 0]


In [25]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score

acc = accuracy_score(y_val, y_pred)
f1 = f1_score(y_val, y_pred, average='macro')

In [28]:
y_result = model.predict(x_test)
print(y_result[:5])

y_result_prob = model.predict_proba(x_test)
print(y_result_prob[:5])

result_prob = pd.DataFrame({
    'result': y_result,
    'prob_0': y_result_prob[:, 0],
    'prob_1': y_result_prob[:, 1],
    'prob_2': y_result_prob[:, 2],
})

print(result_prob)

[2 2 2 0 1]
[[0.01 0.   0.99]
 [0.13 0.13 0.74]
 [0.02 0.1  0.88]
 [0.98 0.02 0.  ]
 [0.06 0.82 0.12]]
    result  prob_0  prob_1  prob_2
0        2    0.01    0.00    0.99
1        2    0.13    0.13    0.74
2        2    0.02    0.10    0.88
3        0    0.98    0.02    0.00
4        1    0.06    0.82    0.12
5        0    0.54    0.42    0.04
6        1    0.03    0.73    0.24
7        0    0.76    0.18    0.06
8        2    0.03    0.27    0.70
9        1    0.01    0.99    0.00
10       2    0.06    0.00    0.94
11       0    0.91    0.04    0.05
12       2    0.02    0.05    0.93
13       1    0.03    0.97    0.00
14       1    0.00    0.86    0.14
15       1    0.29    0.64    0.07
16       1    0.02    0.87    0.11
17       2    0.01    0.00    0.99
18       1    0.00    0.99    0.01
19       0    0.65    0.33    0.02
20       1    0.00    0.88    0.12
21       0    0.89    0.06    0.05
22       1    0.00    0.92    0.08
23       0    0.97    0.02    0.01
24       0    0.86    

In [67]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_diabetes

# diabetes 데이터셋 로드
diabetes = load_diabetes()

x = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = pd.DataFrame(diabetes.target)

# 실기시험 데이터 셋으로 세팅하기
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2023)

x_test = pd.DataFrame(x_test.reset_index())
x_train = pd.DataFrame(x_train.reset_index())
y_train = pd.DataFrame(y_train.reset_index())

# index 컬럼의 이름을 index에서 cust_id로 바꾼거
x_test.rename(columns={'index': 'cust_id'}, inplace=True)
x_train.rename(columns={'index': 'cust_id'}, inplace=True)
# 지금 열 두개 있는데, 열 두개의 이름을 각각 cust_id, target으로 바꿈
y_train.columns = ['cust_id', 'target']

In [68]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)

(353, 11)
(89, 11)
(353, 2)


In [69]:
print(x_train.head())
print(x_test.head())
print(y_train.head())

   cust_id       age       sex       bmi        bp        s1        s2  \
0        4  0.005383 -0.044642 -0.036385  0.021872  0.003935  0.015596   
1      318  0.088931 -0.044642  0.006728  0.025315  0.030078  0.008707   
2      301 -0.001882  0.050680 -0.024529  0.052858  0.027326  0.030001   
3      189 -0.001882 -0.044642 -0.066563  0.001215 -0.002945  0.003070   
4      288  0.070769  0.050680 -0.016984  0.021872  0.043837  0.056305   

         s3        s4        s5        s6  
0  0.008142 -0.002592 -0.031988 -0.046641  
1  0.063367 -0.039493  0.009434  0.032059  
2  0.030232 -0.002592 -0.021395  0.036201  
3  0.011824 -0.002592 -0.020292 -0.025930  
4  0.037595 -0.002592 -0.070209 -0.017646  
   cust_id       age       sex       bmi        bp        s1        s2  \
0      280  0.009016  0.050680  0.018584  0.039087  0.017694  0.010586   
1      412  0.074401 -0.044642  0.085408  0.063187  0.014942  0.013091   
2       68  0.038076  0.050680 -0.029918 -0.040099 -0.033216 -0.02417

In [70]:
print(x_train.info())
print(x_test.info())
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353 entries, 0 to 352
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  353 non-null    int64  
 1   age      353 non-null    float64
 2   sex      353 non-null    float64
 3   bmi      353 non-null    float64
 4   bp       353 non-null    float64
 5   s1       353 non-null    float64
 6   s2       353 non-null    float64
 7   s3       353 non-null    float64
 8   s4       353 non-null    float64
 9   s5       353 non-null    float64
 10  s6       353 non-null    float64
dtypes: float64(10), int64(1)
memory usage: 30.5 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 11 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  89 non-null     int64  
 1   age      89 non-null     float64
 2   sex      89 non-null     float64
 3   bmi      89 non-null     float64
 4   bp      

In [71]:
print(x_train.describe())
print("------------------")
print(x_test.describe())

          cust_id         age         sex         bmi          bp          s1  \
count  353.000000  353.000000  353.000000  353.000000  353.000000  353.000000   
mean   212.634561    0.000804    0.000724    0.000640   -0.000326    0.001179   
std    126.668903    0.047617    0.047673    0.048141    0.046585    0.047891   
min      0.000000   -0.107226   -0.044642   -0.084886   -0.112399   -0.126781   
25%    105.000000   -0.038207   -0.044642   -0.035307   -0.033213   -0.033216   
50%    210.000000    0.005383   -0.044642   -0.006206   -0.005670   -0.002945   
75%    322.000000    0.038076    0.050680    0.030440    0.032201    0.027326   
max    441.000000    0.110727    0.050680    0.170555    0.125158    0.153914   

               s2          s3          s4          s5          s6  
count  353.000000  353.000000  353.000000  353.000000  353.000000  
mean     0.001110   -0.000452    0.000901    0.001446    0.000589  
std      0.048248    0.048600    0.048045    0.047160    0.048122 

In [72]:
print(y_train.describe())  # 연속형 변수 - 회귀분석

          cust_id      target
count  353.000000  353.000000
mean   212.634561  152.943343
std    126.668903   75.324692
min      0.000000   37.000000
25%    105.000000   90.000000
50%    210.000000  141.000000
75%    322.000000  208.000000
max    441.000000  346.000000


In [73]:
# 결측치, 이상치 확인
print(x_train.isnull().sum())
print(x_test.isnull().sum())
print(y_train.isnull().sum())

cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
age        0
sex        0
bmi        0
bp         0
s1         0
s2         0
s3         0
s4         0
s5         0
s6         0
dtype: int64
cust_id    0
target     0
dtype: int64


In [74]:
cust_id = x_test['cust_id'].copy()

x_train = x_train.drop(columns=['cust_id'])
x_test = x_test.drop(columns=['cust_id'])

In [76]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    x_train,
    y_train['target'],
    test_size=0.2,
    random_state=23
)

print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

KeyError: 'target'

In [77]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(random_state=2023)
model.fit(x_train, y_train)

RandomForestRegressor(random_state=2023)

In [78]:
y_pred = model.predict(x_val)

In [79]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

In [80]:
print(mse)
print(r2)

2563.5036816901406
0.5250663004710372


In [81]:
rmse = mse ** 0.5
print(rmse)

50.631054518843875


In [82]:
y_result = model.predict(x_test)
print(y_result)

[186.51 255.92  77.97 185.64 111.14 173.41  89.45 144.33 159.48 191.07
 101.27 129.02  64.59 143.45 135.01  94.2  139.72 114.85 206.35 153.26
  71.56 200.02 115.56 172.34  85.92 175.72 163.62  89.75 196.8  100.58
 100.14 159.16 177.56 186.74 186.92 238.49 231.2  168.   170.88 198.7
  86.86 116.71  96.43 233.38 250.74  92.37  93.85  77.11 230.5  172.38
  98.95  96.71 145.78 171.48 179.94 146.28 220.67  95.5  232.82 163.01
 104.48  83.    92.8   91.78 111.89 116.8  156.7  209.79 247.64 131.45
 169.17 231.46 161.63  78.58 122.61 129.37  68.08 130.35 123.51  73.26
 136.15 156.83 103.81 194.06 192.59 172.71  97.2   86.18 118.42]


In [84]:
result = pd.DataFrame({'cust_id': cust_id, 'target': y_result})
print(result[:5])

   cust_id  target
0      280  186.51
1      412  255.92
2       68   77.97
3      324  185.64
4      101  111.14


In [85]:
# 실기 시험 데이터셋으로 세팅하기

import seaborn as sns
df = sns.load_dataset('tips')

x = df.drop(['tip'], axis=1)
y = df['tip']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2023)

x_train = pd.DataFrame(x_train.reset_index())
x_test = pd.DataFrame(x_test.reset_index())
y_train = pd.DataFrame(y_train.reset_index())